## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embeddings, load one from gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
!pip -q install --upgrade numpy==1.26.4 gensim
# download the data:
# !wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

In [2]:
import numpy as np

with open("./quora.txt", encoding="utf-8") as file:
    data = list(file)

data[:10]

["Can I get back with my ex even though she is pregnant with another guy's baby?\n",
 'What are some ways to overcome a fast food addiction?\n',
 'Who were the great Chinese soldiers and leaders who fought in WW2?\n',
 'What are ZIP codes in the Bay Area?\n',
 'Why was George RR Martin critical of JK Rowling after losing the Hugo award?\n',
 'What can I do to improve my immune system?\n',
 'How is your relationship with your mother in law?\n',
 'How does one get Free PSN codes/Vita Codes?\n',
 'What is your review of osquery?\n',
 'How can I look smart and act smart?\n']

__Tokenization:__ a typical first step for an NLP task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many NLP tasks like tokenization, stemming or part-of-speech tagging.

In [3]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [4]:
# TASK: lowercase everything and extract tokens with tokenizer.
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(line) for line in data]
data_tok = [[tok.lower() for tok in line] for line in data_tok]
data_tok[50]

['what',
 'tv',
 'shows',
 'or',
 'books',
 'help',
 'you',
 'read',
 'people',
 "'",
 's',
 'body',
 'language',
 '?']

In [5]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [6]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings.

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [7]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok,
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

# From gensim docs
# wv: This object essentially contains the mapping between words and embeddings.
# After training, it can be used directly to query those embeddings in various ways.

In [8]:
# now you can get word vectors !
model.get_vector('anything')

array([-3.1745539 ,  1.0391601 ,  1.3215783 ,  1.8967919 ,  1.9700915 ,
        2.3837287 ,  2.155495  , -4.401153  ,  0.42769536,  3.4410315 ,
       -1.5524896 ,  3.768566  ,  4.00241   ,  0.31824434,  1.9351    ,
       -1.5974413 , -0.9466327 , -0.96926004, -0.66592413, -1.3370371 ,
       -1.7577072 , -0.86573917, -2.8423214 , -1.4160128 ,  1.598689  ,
       -1.6830971 , -0.70318633, -0.01765305, -1.0506674 , -0.4984183 ,
       -1.1144117 , -0.0641157 ], dtype=float32)

In [14]:
# or query similar words directly. Go play with it!
model.most_similar('tv')

[('television', 0.8931664824485779),
 ('netflix', 0.7920894622802734),
 ('game', 0.7683335542678833),
 ('anime', 0.7526358366012573),
 ('film', 0.7348645925521851),
 ('video', 0.7239293456077576),
 ('cinema', 0.7218025922775269),
 ('4k', 0.7175209522247314),
 ('ps4', 0.7072328329086304),
 ('movies', 0.6996250748634338)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts.

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

After being downloaded for the first time (or if you manually delete it), the model is saved in the `~/gensim_data` or `%USER_PATH%/gensim_data` directory. This can be checked seting the return_path parameter to True.

In [15]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [23]:
model.most_similar(positive=["science"])

[('physics', 0.8325437903404236),
 ('research', 0.8240416646003723),
 ('biology', 0.8094786405563354),
 ('studies', 0.8021548390388489),
 ('psychology', 0.8005319833755493),
 ('math', 0.7980120182037354),
 ('study', 0.786839485168457),
 ('economics', 0.7681449055671692),
 ('geography', 0.7577798962593079),
 ('education', 0.7489752173423767)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [24]:
words = model.index_to_key[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [31]:
# for each word, compute it's vector with model
word_vectors = [model.get_vector(word) for word in words]
word_vectors = np.array(word_vectors)
word_vectors.shape

(1000, 100)

In [32]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is **P**rincipial **C**omponent **A**nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [36]:
from sklearn.decomposition import PCA

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
pca = PCA(n_components=2)
word_vectors_pca = PCA.fit_transform(pca, word_vectors)
word_vectors_pca = (word_vectors_pca - word_vectors_pca.mean(axis=0)) / word_vectors_pca.std(axis=0)


In [37]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [38]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [39]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p1006', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [41]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca

word_tsne = TSNE().fit_transform(word_vectors)
word_tsne = (word_tsne - word_tsne.mean(axis=0)) / word_tsne.std(axis=0)


In [42]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

figure(id='p1057', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [69]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros

    vector = np.zeros([model.vector_size], dtype='float32')
    tokens = [tok.lower() for tok in tokenizer.tokenize(phrase)]
    count = 0
    for token in tokens:
        try:
            vector += model.get_vector(token) # Sum of vectors so I can divide by count later
            count += 1
        except KeyError:
            pass  # Skip words not in vocabulary

    if count > 0:
        vector /= count
    return vector

In [70]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))
assert np.array_equal(get_phrase_embedding("thisisgibberish"), np.zeros([model.vector_size], dtype='float32')), "corner case for all missing words should be handled as described in the function comments"

In [99]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 5000]

# compute vectors for chosen phrases
phrase_vectors = [get_phrase_embedding(phrase) for phrase in chosen_phrases]
phrase_vectors = np.array(phrase_vectors)
phrase_vectors.shape

(5022, 100)

In [100]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [101]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [102]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

figure(id='p1159', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [103]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    query_vector = get_phrase_embedding(query)
    similarities = np.dot(query_vector, phrase_vectors.T)
    top_k_indices = np.argsort(similarities)[::-1][:k]
    similar_queries = [data[i] for i in top_k_indices]
    return similar_queries
    # return <YOUR CODE: top-k lines starting from most similar>

In [106]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
# assert results[0] == 'How do I get to the dark web?\n'
# assert results[3] == 'What can I do to save the world?\n'

What is the relationship between sound and light?
Can a Canadian business sell American products to Americans?
How can I deal with a violent wife?
What does Shakespeare mean by 'A little more than kin, and less than kind'?
On average, what is the IQ of a doctor?
Which are the best English songs with fast music?
Muslims say not to celebrate self-birthday nor Jesus's birthday on Christmas, then why do Bari Elvis celebrate Eid e-Milad (the Prophet's Muhammad birthday)?
What is the best company to outsource image processing services?
Who are the best girls on Snapchat?
Can you ship sparklers to the US?



In [107]:
find_nearest(query="How does Trump?", k=10)

['Can a Canadian business sell American products to Americans?\n',
 'What is the most insightful poem you came across in Sanskrit poetry?\n',
 'Who are the best girls on Snapchat?\n',
 'What is the relationship between sound and light?\n',
 'What is your review of The Economist (magazine)?\n',
 'How can I deal with a violent wife?\n',
 'Which are the best English songs with fast music?\n',
 'On average, what is the IQ of a doctor?\n',
 'Why did Ambani attend Wharton?\n',
 "What does Shakespeare mean by 'A little more than kin, and less than kind'?\n"]

In [108]:
find_nearest(query="Why don't i ask a question myself?", k=10)

['Can a Canadian business sell American products to Americans?\n',
 'What is the relationship between sound and light?\n',
 'What is the most insightful poem you came across in Sanskrit poetry?\n',
 'On average, what is the IQ of a doctor?\n',
 'How can I deal with a violent wife?\n',
 'Which are the best English songs with fast music?\n',
 "Muslims say not to celebrate self-birthday nor Jesus's birthday on Christmas, then why do Bari Elvis celebrate Eid e-Milad (the Prophet's Muhammad birthday)?\n",
 "What does Shakespeare mean by 'A little more than kin, and less than kind'?\n",
 'Why do astronauts solve white jigsaw puzzles in their training?\n',
 'Who are the best girls on Snapchat?\n']

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize `find_nearest` with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.